In [1]:
# Update driver status
import psycopg2
import sched, time
import numpy as np
import sqlalchemy

In [ ]:
# To connect to a Postgres DB

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="adb_project",
    user="postgres",
    password="topolone"
)

cursor = conn.cursor()

In [ ]:
# Pre - Create and populate table "areas"

query = """
CREATE TABLE areas (
    id SERIAL PRIMARY KEY,
    name VARCHAR(50),
    geom GEOMETRY(POLYGON, 4326)
);

COPY areas(name, geom) FROM 'C:\\Users\\valer\\Desktop\\Advanced_Databases\\adb_project\\ADB_Project\\klm_lecco.kml' WITH (FORMAT 'KML');
"""
cursor.execute(query)

In [ ]:
# Query 1 - Update driver status
query = """
UPDATE drivers
SET driver_status = 2
WHERE driver_id = 8;
"""
cursor.execute(query)

query = """
SELECT driver_status
FROM drivers
WHERE driver_id = 8;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)

In [ ]:
# Query 2 - Insert a new ride
query = """
INSERT INTO rides (
    ride_id, driver_id, user_id, ride_status, request_code,
    pickup_location_lat, pickup_location_lon,
    dropoff_location_lat, dropoff_location_lon,
    request_date, pickup_date, dropoff_date,
    ride_rating, payment_id, passengers_num
) VALUES (
    1, 15, 56, 'TEST', 27164,
    40.89911326259291, -74.19879968,
    40.89119552115688, -73.78466389,
    '2023-05-04', '2023-07-07 20:03', '2023-08-23 07:54',
    4, 1, 2
);
"""
cursor.execute(query)

query = """
SELECT *
FROM rides
WHERE ride_status = "TEST";
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)

In [ ]:
# Query 3 - Insert a new payment
query = """
INSERT INTO rides (
    payment_id, payment_type, fare_amount, promo_code
) VALUES (
    3, 'cash', 415, 'TEST'

);
"""
cursor.execute(query)

query = """
SELECT *
FROM rides
WHERE promo_code = "TEST";
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)

In [ ]:
# Query 3 - Update the rating of a driver
query = """
UPDATE drivers
SET
    rating = (rating * nratings + 5) / (nratings + 1),
    nratings = nratings + 1
WHERE driver_id = 8;
"""
cursor.execute(query)

query = """
SELECT rating
FROM drivers
WHERE driver_id = 8;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)

In [ ]:
# Query 4 - Update the rating of a user
query = """
UPDATE users
SET
    rating = (rating * nratings + 5) / (nratings + 1),
    nratings = nratings + 1
WHERE user_id = 8;
"""
cursor.execute(query)

query = """
SELECT rating
FROM users
WHERE user_id = 8;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)

In [ ]:
# Query 5 - Insert a new user
query = """
INSERT INTO users (
    user_id, first_name, last_name, date_of_birth, residence, email,
    phone_number, passw, account_status, rating, nrating
) VALUES (
    4, 'TEST', 'Johnston', '11/12/1968', 'Lake Cynthia',
    'amberhuff@example.org', '9237740873', 'pX@h6G_xFm', 'disabled', 3, 57
    );

"""
cursor.execute(query)

query = """
SELECT *
FROM users
WHERE first_name = "TEST";
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)

In [ ]:
# Query 6 - Insert a new driver
query = """
INSERT INTO drivers (
    driver_id, first_name, last_name, driver_status, date_of_birth,
    place_of_birth, place_of_residence, nationality, email, phone_number,
    licence_id, taxi_licence_id, rating, vehicle_id, join_date, passw, nrating
) VALUES (
    6, 'TEST', 'Park', 'disabled', '1998-03-01',
    'Melissaside', 'Chelseaview', 'Ireland', 'fischeramy@example.org', '+1-996-977-8599x71035',
    3556, 32923, 2, 6, '2022-06-23', '6A85+UQtE)', 57
);
"""
cursor.execute(query)

query = """
SELECT *
FROM drivers
WHERE first_name = "TEST";
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)

In [ ]:
# Query 7 - Insert a new vehicle
query = """
INSERT INTO vehicles (
    vehicle_id, licence_plate_num, manufacturer, model, manifacture_year, 
    car_policy_num, car_type, fuel, seats_num, kids_seats_num, 
    wheelchair_seat
) VALUES (
    1, 'TEST', 'Evans, Wilson and Joseph', 'allow', 2017,
    'MPIN52883961303597', 'high', 'electric', 4, 0, FALSE
);
"""
cursor.execute(query)

query = """
SELECT *
FROM vehicles
WHERE licence_plate_num = "TEST";
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)

In [ ]:
# Query 8 - Number of rides in the last 15 minutes in each area

query = """
SELECT
    a.name AS area_name,
    COUNT(r.id) AS rides_count
FROM
    areas a
LEFT JOIN (
    SELECT
        id,
        latitude,
        longitude,
        ST_SetSRID(ST_MakePoint(longitude, latitude), 4326) AS point,
        dropoff_date
    FROM
        rides
    WHERE
        dropoff_date >= NOW() - INTERVAL '15 minutes'
) r ON ST_Within(r.point, a.geom)
GROUP BY
    a.name;
"""

# Explained:
# 1. The subquery returns 'rides' rows of the last 15 minutes
# 2. Left join to check the belonging of each point (why left? Because if there are points outisde of all areas, they are just not considered)
# 3. Group by to count

cursor.execute(query)
result = cursor.fetchall()
print(result)

In [ ]:
# Query 9 - Daily ETL
query = """
COPY rides TO 'C:\\Users\\valer\\Downloads\\rides_export.csv' WITH CSV HEADER;
"""
cursor.execute(query)

query = """
DELETE FROM rides;
"""
cursor.execute(query)

query = """
SELECT *
FROM rides
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)

In [ ]:
cursor.close()
conn.close()